In [1]:
import pygame

BLACK = (0,0,0)

class Paddle(pygame.sprite.Sprite):
    def __init__(self, color, width, height):
        super().__init__()

        self.image = pygame.Surface([width, height])
        self.image.fill(BLACK)
        self.image.set_colorkey(BLACK)

        pygame.draw.rect(self.image, color, [0, 0, width, height])

        self.rect = self.image.get_rect()

    def moveUp(self, pixels):
        self.rect.y -= pixels

        if self.rect.y < 0:
            self.rect.y = 0

    def moveDown(self, pixels):
        self.rect.y += pixels

        if self.rect.y > 400:
            self.rect.y = 400
            
    def get_paddle(self):
        return self.rect.y

import pygame
from random import randint

BLACK = (0, 0, 0)

class Ball(pygame.sprite.Sprite):

    def __init__(self, color, width, height):
        super().__init__()

        self.image = pygame.Surface([width, height])
        self.image.fill(BLACK)
        self.image.set_colorkey(BLACK)

        pygame.draw.rect(self.image, color, [0, 0, width, height])

        self.velocity = [4, randint(-6,6)]

        self.rect = self.image.get_rect()

        self.tx = 0
        self.ty = 0
        
    def ang(self):
        return self.ty / self.tx
    
    def update(self):
        t_x = self.rect.x
        t_y = self.rect.y
        self.rect.x += self.velocity[0]
        self.rect.y += self.velocity[1]
        self.tx = t_x - self.rect.x
        self.ty = t_y - self.rect.y

    def bounceA(self):
        self.velocity[0] = -self.velocity[0]
        self.velocity[1] = randint(-6,6)
        self.rect.x = 11
        
    def bounceB(self):
        self.velocity[0] = -self.velocity[0]
        self.velocity[1] = randint(-6,6)
        self.rect.x = 680
        
    def get_ball(self):
        return self.rect.x, self.rect.y

        
from gym.spaces import Discrete, Box
import random

class pygametest(pygame.sprite.Sprite):
    
    def __init__(self):
        super().__init__()
        pygame.init()
        
        #-----------------------------------------------
        
        self.action_space = Discrete(2)  # 0,1, 두가지 액션, 위아래
        
        # 상태값을 나타내고, 현 코드에서는 0부터 100까지를 나타낼 것
        self.observation_space = Box(low=0, high=5000, shape=(4,), dtype=int) 
#         self.target = Target()
        self.state = (0, 0, 0, 0)
        self.episodes = 5000
        self.hit = 0
        self.count = 0
        self.memory = []
        self.fire_count = 0
        self.rad = []
        
        
        
        #-----------------------------------------------
        # Define some colors
        self.BLACK = (0,0,0)
        self.WHITE = (255,255,255)

        # Open a new window
        self.size = (700, 500)
        self.screen = pygame.display.set_mode(self.size)
        pygame.display.set_caption("Pong")

        self.paddleA = Paddle(self.WHITE, 10, 100)
        self.paddleA.rect.x = 0
        self.paddleA.rect.y = 200

        self.paddleB = Paddle(self.WHITE, 10, 100)
        self.paddleB.rect.x = 690
        self.paddleB.rect.y = 200

        self.balls = []
        

        self.ball = Ball(self.WHITE,10,10)
        self.ball.rect.x = 345
        self.ball.rect.y = 195

        #This will be a list that will contain all the sprites we intend to use in our game.
        self.all_sprites_list = pygame.sprite.Group()

        # Add the car to the list of objects
        self.all_sprites_list.add(self.paddleA)
        self.all_sprites_list.add(self.paddleB)
        self.all_sprites_list.add(self.ball)

        # The clock will be used to control how fast the screen updates
        self.clock = pygame.time.Clock()

        #Initialise player scores
        self.scoreA = 0
        self.scoreB = 0
        
        self.bounce_state = (0, 0, 0, 0)
        self.reward_point = 0

    def bouncePaddle(self, a, b, x, y):
        self.bounce_state = a, b, x, y
        
    def step(self, act):
        self.episodes -= 1
        self.count = 0
        done = False
        
#         while not done:
            

        self.screen.fill(BLACK)
        #Draw the net
        pygame.draw.line(self.screen, self.WHITE, [349, 0], [349, 500], 5)

        # --- Main event loop
        self.event = pygame.event.poll()
        if self.event.type == pygame.QUIT: # If user clicked close
              pygame.quit()
        elif self.event.type==pygame.KEYDOWN:
                if self.event.key==pygame.K_x: #Pressing the x Key will quit the game
                     pygame.quit()

        keys = pygame.key.get_pressed()
        if act == 0: # keys[pygame.K_w]: # 
            self.paddleA.moveUp(5)
        if act == 1: # keys[pygame.K_s]: #
            self.paddleA.moveDown(5)


#         rand = random.randrange(2)
        if keys[pygame.K_UP]:#rand == 0: # 
            self.paddleB.moveUp(5)
        if keys[pygame.K_DOWN]:# rand == 1: # 
            self.paddleB.moveDown(5)    

        self.all_sprites_list.update()

        # a 학습
        if self.ball.rect.x>=690:
            self.scoreA+=1
#             self.count = 1
            if self.reward_point == 1:
#                 self.count = 10
                info = {}
                self.reward_point = 0
                
            
            self.ball.velocity[0] = -self.ball.velocity[0]
            print("득점 : ", self.paddleA.get_paddle(), self.ball.get_ball())
        if self.ball.rect.x<=0:
            self.scoreB+=1
#             done = True
            self.count = -50
            info = {}
            p1 = self.paddleA.get_paddle()
            p2 = self.paddleB.get_paddle()
            b_x, b_y = self.ball.get_ball()
            self.reward_point = 0
            self.bouncePaddle(p1, p2, b_x, b_y)
            self.ball.velocity[0] = -self.ball.velocity[0]
            print("짐 : ", self.paddleA.get_paddle(), self.ball.get_ball())
            done = True
        if self.ball.rect.y>490:
            self.ball.velocity[1] = -self.ball.velocity[1]
            self.ball.rect.y = 490
        if self.ball.rect.y<0:
            self.ball.velocity[1] = -self.ball.velocity[1]  
            self.ball.rect.y = 0


        if pygame.sprite.collide_mask(self.ball, self.paddleA):
            print("반사 : ", self.paddleA.get_paddle(), self.ball.get_ball())
            self.count = 20
            info = {}
            p1 = self.paddleA.get_paddle()
            p2 = self.paddleB.get_paddle()
            b_x, b_y = self.ball.get_ball()
            self.reward_point = 1
            self.bouncePaddle(p1, p2, b_x, b_y)
            self.ball.bounceA()
#             return (p1, p2, b_x, b_y), self.count, done, info

        if pygame.sprite.collide_mask(self.ball, self.paddleB):
            print("막음 : ", self.paddleA.get_paddle(), self.ball.get_ball())
            self.ball.bounceB()
            self.reward_point = 0
            self.bounce_state = (0, 0, 0, 0)

        self.all_sprites_list.draw(self.screen) 

        font = pygame.font.Font(None, 74)
        text = font.render('AI :'+ str(self.scoreA), 1, self.WHITE)
        self.screen.blit(text, (150,10))
        text = font.render('User :' + str(self.scoreB), 1, self.WHITE)
        self.screen.blit(text, (420,10))

        pygame.display.update()

        self.clock.tick(60)


        if self.episodes <= 0:
            done = True

        info = {}
        p1 = self.paddleA.get_paddle()
        p2 = self.paddleB.get_paddle()
        b_x, b_y = self.ball.get_ball()
        return (p1, self.ball.velocity[0], b_x, b_y), self.count, done, info


    def reset(self):
        self.state = (0,0)
        self.episodes = 5000
        self.count = 0
        
        self.paddleA.rect.x = 0
        self.paddleA.rect.y = 200

        self.paddleB.rect.x = 690
        self.paddleB.rect.y = 200
        
        self.bounce_state = (0, 0, 0, 0)

        self.ball.rect.x = 345
        self.ball.rect.y = 195
        self.ball.velocity = [4, randint(-6,6)]
        return 0,0,0,0

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def build_model(input_shape, action_space):
    X_input = Input(input_shape)

    X = Dense(16, input_shape=input_shape, activation="relu", kernel_initializer='he_uniform')(X_input)
    X = Dense(16, activation="relu", kernel_initializer='he_uniform')(X)
    X = Dense(32, activation="linear", kernel_initializer='he_uniform')(X)
    X = Dense(16, activation="relu", kernel_initializer='he_uniform')(X)
    X = Dense(action_space, activation="linear", kernel_initializer='he_uniform')(X)

    model = Model(inputs = X_input, outputs = X, name='CartPoleDQNmodel')
    model.compile(loss="mse", optimizer=RMSprop(lr=0.00025, rho=0.95, epsilon=0.01), metrics=["accuracy"])   
    model.summary()
    return model


In [4]:
import time

class DQNAgent:
    def __init__(self):
        self.env = pygametest()
        self.state_size = self.env.observation_space.shape[0] # 4 차원
        print(self.state_size)
        self.action_size = self.env.action_space.n#2
        self.EPISODES = 500
        self.memory = deque(maxlen=3000)
            
        self.gamma = 0.95   
        self.epsilon = 1.0 
        self.epsilon_min = 0.001
        self.epsilon_decay = 0.9999
        self.batch_size = 64
        self.train_start = 1000

#         self.model = build_model(input_shape=(self.state_size,), action_space = self.action_size)
        self.model2 = load_model("Not_show_train2.h5")
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > self.train_start:
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay             
        
    def act(self, state):
        if np.random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            return np.argmax(self.model.predict(state))
        

    def replay(self):
        if len(self.memory) < self.train_start:
            return
        minibatch = random.sample(self.memory, min(len(self.memory), self.batch_size))
        

        state = np.zeros((self.batch_size, self.state_size))
        next_state = np.zeros((self.batch_size, self.state_size))
        action, reward, done = [], [], []

        for i in range(self.batch_size):
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            done.append(minibatch[i][4])

        target = self.model.predict(state)
        target_next = self.model.predict(next_state)

        for i in range(self.batch_size):
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                target[i][action[i]] = reward[i] + self.gamma * (np.amax(target_next[i]))

        self.model.fit(state, target, batch_size=self.batch_size, verbose=0)

    def load(self, name):
        self.model = load_model(name)

    def save(self, name):
        self.model.save(name)

    def run(self):
        re_count = 0

        for e in range(500):
            state = self.env.reset() 
            state = np.reshape(state, [1, self.state_size])
            
#             state2 = state
#             state2[0][2] = 200 - state[0][2]
            
            done = False 
            i = 0
            
            while not done:
                # 학습시킬 A패들
#                 action = self.act(state)
                
                # 학습시킬 B패들
#                 action2 = self.act(state)
                # 학습중
                # 학습된 A패들
                action = np.argmax(self.model2.predict(state))
                # 학습된 B패들
#                 action2 = np.argmax(self.model2.predict(state))
            
                next_state, reward, done, _ = self.env.step(action)
#                 print(reward)
#                 print(type(reward))
                next_state = np.reshape(next_state, [1, self.state_size])
    
#                 time.sleep(0.2)
#                 if not done or i == self.env.episodes-1:
#                     reward = reward
#                 elif done and i != self.env.episodes-1:
#                     reward = -100
                
                # A패들 remember
#                 if reward != 0:
# #                     print(reward)
#                     self.remember(state, action, reward, next_state, done)
#                 # B패들 remember
# #                 self.remember(state, action2, reward, next_state, done)
                
                state = next_state

#                 self.replay()

    def test(self):
        self.load("show_train.h5")
        for e in range(self.EPISODES):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size])
            done = False
            i = 0
            while not done:
#                 self.env.render()
                action = np.argmax(self.model2.predict(state))
                next_state, reward, done, _ = self.env.step(action)
                state = np.reshape(next_state, [1, self.state_size])
                i += 1
                if done:
                    print("episode: {}/{}, score: {}".format(e, self.EPISODES, i))
                    break
                self.replay()

In [5]:
agent = DQNAgent()
agent.run()

4
막음 :  270 (681, 370)
반사 :  220 (8, 276)
막음 :  310 (683, 425)
반사 :  165 (8, 220)


error: video system not initialized